# Back Propagation : 역전파 알고리즘
- 수치적 기울기 : 미분의 정의로부터 극한 연산을 근사해 수치적 기울기를 구할 수 있다.
    > 입실론의 값이 충분히 작다면, 수치적 기울기를 미분 값으로 사용 가능.<br>
    > 각 스칼라 변수를 조금씩 바꾸어 대입해보면서 수치적 기울기를 구한다.<br>
    > N개의 매개 변수로 미분하기 위해 (N+1)번 더 손실함수를 평가해야한다.<br>

- 심층 신경망의 수치적 기울기 
    > Gradient Descent 한 스텝 계산을 위한 연산은 N(N + 1)번의 곱하기연산이다.<br>
    > 10만개의 파라미터를 가진 경우 무려 100억 회가 필요하므로, 대책이 필요하다.

- Sigmoid 함수의 미분
    > sigmoid(x)(1-sigmoid(x))로 표현할 수 있다.<br>


# 그래서 역전파 알고리즘(BP)이란?
<br>
    - 학습 데이터로 정방향 연산을 하여 Loss를 구한다. <br>
    - 정방향 연산 시, 계층 별로 BP에 필요한 중간결과를 저장한다.<br>
    - Loss를 각 파라미터로 미분한다. 연쇄 법칙(역방향 연산)을 이용한다.<br>
        - 마지막 계층부터 하나씩 이전 계층으로 연쇄적으로 계산한다.<br>
        - 역방향 연산 시, 정방향 연산에서 저장한 중간 결과를 사용한다.

> 미분의 연쇄법칙과 각 함수의 수식적 미분을 이용하면, 단 한번의 손실함수 평가로 미분을 구할 수 있다. 단, 중간결과를 저장해야 하므로 메모리를 추가로 사용한다.

### 활성함수의 개선 - ReLU
- 양수랴면, 입력에 관계 없이 동일한 미분 값(1)이 나오게 하여 기울기 소실 문제를 해결.<br>

# 수치 미분을 이용한 심층 신경망 학습

### import modules

In [46]:
import time
import numpy as np

### 유틸리티 함수

In [58]:
epsilon = 0.0001

def _t(x):
    return np.transpose(x)

def _m(A, B):
    return np.matmul(A, B)

### Sigmoid 함수 구현

In [59]:
class Sigmoid:
    def __init__(self):
        self.last_o = 1

    def __call__(self, x):
        self.last_o = 1.0 / (1.0 + np.exp(-x))
        return self.last_o

    def grad(self): # sigmoid(x)(1 - sigmoid(x))
        return self.last_o * (1.0 - self.last_o)

### MSE 구현

In [60]:
class MeanSquaredError: # 1/2 * mean((h - y)^2)  --> h - y
    def __init__(self):
        self.dh = 1
        self.last_diff = 1

    def __call__(self, h, y):
        self.last_diff = h - y
        return 1 / 2 * np.mean(np.square(self.last_diff))

    def grad(self):
        return self.last_diff

### Dense Layer 구현

In [61]:
class Dense:
    def __init__(self, W, b, a_obj):
        self.W = W
        self.b = b
        self.a = a_obj()
        
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)
        self.dh = np.zeros_like(_t(self.W))
        
        self.last_x = np.zeros((self.W.shape[0]))
        self.last_h = np.zeros((self.W.shape[1]))
        

    def __call__(self, x):
        self.last_x = x
        self.last_h = _m(_t(self.W), x) + self.b
        return self.a(self.last_h)

    def grad(self): # dy/dh = W
        return self.W * self.a.grad()

    def grad_W(self, dh):
        grad = np.ones_like(self.W)
        grad_a = self.a.grad()
        for j in range(grad.shape[1]): # dy/dw = x
            grad[:, j] = dh[j] * grad_a[j] * self.last_x
        return grad

    def grad_b(self, dh): # dy/db = 1
        return dh * self.a.grad()

In [48]:
'''class Dense:
    def __init__(self, W, b, a):
        self.W = W
        self.b = b
        self.a = a

        self.dW = np.zeros_like(self.W) # w를 미분한 것
        self.db = np.zeros_like(self.b) # b를 미분한 것
        
    def __call__(self, x):
        return self.a(_m(_t(self.W), x) + self.b) 
        # matmul((input * output)T , i * 1) + o * 1'''

- 제대로 동작하지 않아 다른 함수 정의 (추후 보완 예정)

### 심층신경망 구현

In [49]:
'''class DNN:
    def __init__(self, hidden_depth, num_neuron, num_input, num_output, activation=sigmoid):
        def init_var(i, o):
            return np.random.normal(0.0, 0.01, (i, o)), np.zeros((o, ))
        self.sequence = list()
        # First hidden layer
        W, b = init_var(num_input, num_neuron)
        self.sequence.append(Dense(W, b, activation))

        # Hidden layers
        for _ in range(hidden_depth - 1):
            W, b = init_var(num_neuron, num_neuron)
            self.sequence.append(Dense(W, b, activation))

        # Output layer
        W, b = init_var(num_neuron, num_output)
        self.sequence.append(Dense(W, b, activation))

    def __call__(self, x):
        for layer in self.sequence:
            x = layer(x)
        return x

    def calc_gradient(self, x, y, loss_func):
        def get_new_sequence(layer_index, new_layer): # 값을 수정해서 새로운 sequence를 만들어주는 function
            new_sequence = list()
            for i, layer in enumerate(self.sequnece):
                if i == layer_index:
                    new_sequence.append(new_layer)
                else:
                    new_sequence.append(layer)
            return new_sequence

        def eval_sequence(x, sequence): # 새로 만든 sequence로 평가를 하는 function
            for layer in sequence:
                x = layer(x)
            return x
        
    
        loss = loss_func(self(x), y)

        for layer_id, layer in enumerate(self.sequence):
            for w_i, w in enumerate(layer.W):
                for w_j, ww in enumerate(w): # 행과 열 모두 iteration
                    W = np.copy(layer.W)
                    W[w_i][w_j] = ww + epsilon

                    new_layer = Dense(W, layer.b, layer.a)
                    new_seq = get_new_sequence(layer_id, new_layer)
                    h = eval_sequence(x, new_seq)
                    num_grad = (loss_func(h, y) - loss) / epsilon
                    # Numerical Gradients : (f(x + eps) - f(x)) / eps
                    layer.dW[w_i][w_j] = num_grad
                    
            for b_i, bb in enumerate(layer.b): 
                # bias는 vector이기 때문에 iteration은 1번만 돌려도 됨.
                b = np.copy(layer.b)
                b[b_i] = bb + epsilon

                new_layer = Dense(layer.W, b, layer.a)
                new_seq = get_new_sequence(layer_id, new_layer)
                h = eval_sequence(x, new_seq)
                num_grad = (loss_func(h, y) - loss) / epsilon
                # Numerical Gradients : (f(x + eps) - f(x)) / eps
                layer.db[b_i] = num_grad
              

        return loss'''

- 제대로 동작하지 않아 다른 함수 정의

In [62]:
class DNN:
    def __init__(self, hidden_depth, num_neuron, input, output, activation=sigmoid):
        def init_var(i, o):
            return np.random.normal(0.0, 0.01, (i, o)), np.zeros((o,))

        self.sequence = list()
        # First hidden layer
        W, b = init_var(input, num_neuron)
        self.sequence.append(Dense(W, b, activation))

        # Hidden Layers
        for index in range(hidden_depth):
            W, b = init_var(num_neuron, num_neuron)
            self.sequence.append(Dense(W, b, activation))

        # Output Layer
        W, b = init_var(num_neuron, output)
        self.sequence.append(Dense(W, b, activation))

    def __call__(self, x):
        for layer in self.sequence:
            x = layer(x)
        return x

    def calc_gradient(self, loss_obj):
        loss_obj.dh = loss_obj.grad()
        self.sequence.append(loss_obj)
        
        # back-prop loop
        for i in range(len(self.sequence) - 1, 0, -1):
            l1 = self.sequence[i]
            l0 = self.sequence[i - 1]
            
            l0.dh = _m(l0.grad(), l1.dh)
            l0.dW = l0.grad_W(l1.dh)
            l0.db = l0.grad_b(l1.dh)
        
        self.sequence.remove(loss_obj)

### 정리
- 1) Numerical Gradient를 구하기 위해서는 기준이 되는 loss를 구해야함.
- 2) 각각의 학습을 하고자하는 스칼라들을 epsilon만큼 옮겨서 loss function을 새로 구하고,
- 3) 기준이 되는 loss function이랑 차이를 구해준다.
- 4) epsilon 만큼 이동하였으니 epsilon으로 나눠주면, Numerical Gradient 정의에 의해서 구하게 되고 해당 스칼라 위치에 Gradient 저장

### 경사하강 학습법

In [63]:
def gradient_descent(network, x, y, loss_obj, alpha=0.01):
    loss = loss_obj(network(x), y)  # Forward inference
    network.calc_gradient(loss_obj)  # Back-propagation
    for layer in network.sequence:
        layer.W += -alpha * layer.dW
        layer.b += -alpha * layer.db
    return loss

### 동작 테스트

In [64]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

t = time.time()
dnn = DNN(hidden_depth=5, num_neuron=32, input=10, output=2, activation=Sigmoid)
loss_obj = MeanSquaredError()
for epoch in range(100):
    loss = gradient_descent(dnn, x, y, loss_obj, alpha=0.01)
    print('Epoch {}: Test loss {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

Epoch 0: Test loss 0.2047870444406289
Epoch 1: Test loss 0.2024852265882107
Epoch 2: Test loss 0.2002110775603565
Epoch 3: Test loss 0.1979650190753927
Epoch 4: Test loss 0.19574742495064051
Epoch 5: Test loss 0.19355862200604426
Epoch 6: Test loss 0.19139889111581532
Epoch 7: Test loss 0.1892684683933229
Epoch 8: Test loss 0.18716754649436362
Epoch 9: Test loss 0.185096276024002
Epoch 10: Test loss 0.183054767032393
Epoch 11: Test loss 0.18104309058534576
Epoch 12: Test loss 0.17906128039585895
Epoch 13: Test loss 0.17710933450342906
Epoch 14: Test loss 0.17518721698858827
Epoch 15: Test loss 0.17329485971085162
Epoch 16: Test loss 0.17143216405903
Epoch 17: Test loss 0.16959900270367623
Epoch 18: Test loss 0.1677952213422702
Epoch 19: Test loss 0.16602064042859202
Epoch 20: Test loss 0.16427505687858135
Epoch 21: Test loss 0.16255824574581576
Epoch 22: Test loss 0.16086996186055466
Epoch 23: Test loss 0.1592099414270889
Epoch 24: Test loss 0.15757790357488943
Epoch 25: Test loss 0.15

### Hidden_depth를 10으로 증가시켜보자

In [65]:
x = np.random.normal(0.0, 1.0, (10,))
y = np.random.normal(0.0, 1.0, (2,))

t = time.time()
dnn = DNN(hidden_depth=10, num_neuron=32, input=10, output=2, activation=Sigmoid)
loss_obj = MeanSquaredError()
for epoch in range(100):
    loss = gradient_descent(dnn, x, y, loss_obj, alpha=0.01)
    print('Epoch {}: Test loss {}'.format(epoch, loss))
print('{} seconds elapsed.'.format(time.time() - t))

Epoch 0: Test loss 1.332709435842509
Epoch 1: Test loss 1.3177567856052912
Epoch 2: Test loss 1.3030070049852824
Epoch 3: Test loss 1.2884824351838535
Epoch 4: Test loss 1.2742033237473076
Epoch 5: Test loss 1.2601877066044778
Epoch 6: Test loss 1.2464513388166418
Epoch 7: Test loss 1.233007671788581
Epoch 8: Test loss 1.219867873198329
Epoch 9: Test loss 1.2070408847720233
Epoch 10: Test loss 1.194533512277451
Epoch 11: Test loss 1.1823505417228335
Epoch 12: Test loss 1.1704948756894162
Epoch 13: Test loss 1.158967683944489
Epoch 14: Test loss 1.1477685629137768
Epoch 15: Test loss 1.136895699175177
Epoch 16: Test loss 1.1263460328095978
Epoch 17: Test loss 1.1161154171565775
Epoch 18: Test loss 1.106198772229416
Epoch 19: Test loss 1.096590229713919
Epoch 20: Test loss 1.0872832680838163
Epoch 21: Test loss 1.0782708369006286
Epoch 22: Test loss 1.069545469819889
Epoch 23: Test loss 1.0610993861987157
Epoch 24: Test loss 1.05292458149559
Epoch 25: Test loss 1.0450129068785012
Epoch 2